<a href="https://colab.research.google.com/github/Tommmster/6620_/blob/master/CartPole_tryout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!apt-get install x11-utils
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!pip install pyglet==1.5.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libxxf86dga1
Suggested packages:
  mesa-utils
The following NEW packages will be installed:
  libxxf86dga1 x11-utils
0 upgraded, 2 newly installed, 0 to remove and 21 not upgraded.
Need to get 209 kB of archives.
After this operation, 711 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxxf86dga1 amd64 2:1.1.4-1 [13.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 x11-utils amd64 7.7+3build1 [196 kB]
Fetched 209 kB in 2s (126 kB/s)
Selecting previously unselected package libxxf86dga1:amd64.
(Reading database ... 146981 files and directories currently installed.)
Preparing to unpack .../libxxf86dga1_2%3a1.1.4-1_amd64.deb ...
Unpacking libxxf86dga1:amd64 (2:1.1.4-1) ...
Selecting previously unselected package x11-utils.
Preparing to unpack .../x11-utils_7.7+3build1_amd64.de

In [3]:
import pyvirtualdisplay


_display = pyvirtualdisplay.Display(visible=False,  # use False with Xvfb
                                    size=(1400, 900))
_display.start()
print(_display)


In [4]:
import matplotlib.pyplot as plt

# preparar el display
%matplotlib notebook
fig = plt.figure()
ax = fig.add_subplot(111)
fig.show()



<IPython.core.display.Javascript object>

In [5]:
import itertools
import numpy as np
import pandas as pd
from collections import defaultdict

In [33]:
from math import floor
class Values:
    def __init__(self, numActions, alpha, discount_factor, initial):
        self.values = defaultdict(lambda : np.full(numActions, initial, dtype='float'))
        self.alpha = alpha
        self.discount = discount_factor

    def get(self, state, action):
        return self.values[self.key_for(state)][action]

    def put(self, state, action, value):
        self.values[self.key_for(state)][action] = value
        return self

    def update(self, state, action, new_state, new_action, reward):
      alpha = self.alpha
      df = self.discount_factor

      old = self.get(state, action)
      new = self.get(new_state, new_action)

      updated = old + old * (1 - alpha) + alpha * (reward + df * new)
      
      self.put(state, action, updated)
    
    def len(self):
        return len(self.values)

    def argmax_for(self, state):
        actions = self.values[self.key_for(state)]
        return actions.argmax()
    
    def key_for(self, state):
      print(state)
      return tuple(state)


class CoarseValues(Values): 
    def __init__(self, numActions, initial = 0.0):
        super().__init__(numActions, initial)
    
    def _partition_position(self, position):
        return floor(position * 100) 
    
    def _partition_velocity(self, velocity):
        
        return floor(velocity * 10)

    def _partition_angle(self, angle):
        return self._partition_position(angle)

    def _partition_angular_velocity(self, av):
        return self._partition_velocity(av)


    def key_for(self, state):
        position, velocity, angle, angular_velocity = state

        return super().key_for (
            [
                self._partition_position(position), self._partition_velocity(velocity),
                self._partition_angle(angle), self._partition_angular_velocity(angular_velocity)
            ]
        )


In [7]:
%matplotlib inline
import gym
import time
from IPython import display as ipythondisplay


env = gym.make('CartPole-v0')
# observation space: posición, velocidad, ángulo, velocidad de rotación

env.reset()
for i in range(20):
  pass
  #screen = env.render("rgb_array")
  #plt.imshow(screen)
  #ipythondisplay.clear_output(wait=True)
  #ipythondisplay.display(plt.gcf())
  #result = env.step(env.action_space.sample())
  #print(result)
 


In [38]:
def make_epsilon_greedy_policy(Q, epsilon, nA):
    """
    Crea una política epsilon-greedy basado en una q-función (función de valor estado-acción) y un epsilon dados.
    
    Argumentos:
        Q: un diccionario que mapea cada estado/observación s a un array de numpy Q[s] = array([v_0, v_1, ... , v_nA]) de longitud nA
        que para un índice a del array contiene el valor v_a de tomar la acción a en el estado s. 
        (en nuestra notación de la clase q(s,a))
         
        epsilon: probabilidad de seleccionar una acción aleatoria (obliga a explorar), valor entre 0 y 1.
        
        nA: número de acciones en el entorno
    
    Retorna:
        Una función que dada una observación como argumento, retorna una política (un array de numpy de longitud nA)
        con probabilidades para cada acción. La política es tal que toma la mejor acción según Q con probabilidad (1-epsilon)
        y toma una acción al azar con probabilidad epsilon 
    """
    
    def policy_fn(observation):
        A = np.ones(nA, dtype=float) * epsilon / nA
        best_action = Q.argmax_for(observation)
        A[best_action] += (1.0 - epsilon)
        return A

    return policy_fn

def sarsa(env, num_episodes, make_Q, discount_factor=0.95, alpha=0.5, epsilon=0.1):
    """
    Algoritmo SARSA: Control por TD (diferencias temporales) on-policy. Encuentra la política epsilon-greedy óptima.
    
    Argumentos:
        env: Ambiente de OpenAI.
        num_episodes: Número de episodios para simular.
        discount_factor: factor de descuento gama.
        alpha: factor de aprendizaje de TD.
        epsilon: Probabilidad de tomar una acción aleatoria. Entre 0 y 1.
    
    Retorna:
        Una tupla (Q, stats).
        A tuple (Q, stats).
        Q es la función de valor estado-acción óptima, un diccionario que mapea estado -> array de valores para cada acción.
        stats es un objeto EpisodeStats con dos arrays numpy para longitud de episodios y longitud de recompensas.
    """
    # La función de valor estado-acción final (q-función final)
    #  un diccionario que mapea estado -> array de valores para cada acción.
    # Q = defaultdict(lambda: np.zeros(env.action_space.n))
    
    Q = make_Q(env.action_space.n, alpha, discount_factor, 1 / (1 - alpha))
    
    # la política que seguimos
    policy = make_epsilon_greedy_policy(Q, epsilon, env.action_space.n)
    
    all_rewards = []
    
    get_next_action = lambda ps : np.random.choice(np.arange(len(ps)), p = ps)
    for i_episode in range(num_episodes):
        
        # Resetear el entorno y tomar primera acción
        state = env.reset()
        action_probs = policy(state)

        action = get_next_action(action_probs)
        rewards = []

        # loop en el entorno
        for t in itertools.count():
            
            next_state, reward, done, _ = env.step(action)
            rewards.append(reward)

            # print("{}: {} ->{} ".format(t, state, next_state))
            # elegir próxima acción 
            next_action_probs = policy(next_state)
            next_action = get_next_action(next_action_probs)
            
            
            # Atualización TD:
            old = Q.get(state, action)
            new = Q.get(next_state, next_action)

            updated = old + old * (1 - alpha) + alpha * (reward + discount_factor * new)
            Q.put(state, action, updated)
    
            if done:
                all_rewards.append(np.sum(rewards))
                print("-----")
                break
                
            action = next_action
            state = next_state
         
    
    return Q, None, np.mean(all_rewards)

    # Episode Termination:
        * Pole Angle is more than 12 degrees (0.209 radians).
        * Cart Position is more than 2.4 (center of the cart reaches the edge of
        the display).
        * Episode length is greater than 200.
        
    # Solved Requirements:
        Considered solved when the average return is greater than or equal to
        195.0 over 100 consecutive trials.

In [ ]:
Q, _, mean_rewards = sarsa(env, 1, lambda actions, alpha, discount, i:  Values(actions, alpha i))

mean_rewards

In [ ]:
Q, _, mean_rewards = sarsa(env, 100, lambda actions, i:  CoarseValues(actions, i))

mean_rewards

In [41]:
class CoarseGrid(Values): 
    
    def __init__(self, numActions, bias, to_coarse, initial = 0.0):
        super().__init__(numActions, initial)
        self.bias = bias
        self.coarse_position = to_coarse[0]
        self.coarse_velocity = to_coarse[1]
        self.coarse_angle = to_coarse[2]
        self.coarse_angular_velocity = to_coarse[3]
    
    def key_for(self, state):
        position, velocity, angle, angular_velocity = state + self.bias

        return super().key_for (
            [
                self.coarse_position(position), self.coarse_velocity(velocity),
                self.coarse_angle(angle), self.coarse_angular_velocity(angular_velocity)
            ]
        )

class CMAC:
    def __init__(self, grids):
        self.grids = grids
    
    def get(self, state, action):
        return sum([g.get(state, action) for g in self.grids])

    def update(self, state, action, new_state, new_action, reward):
        for g in self.grids:
            g.update(state, action, new_state, new_action, reward)

In [42]:
from math import floor
coarse_position = lambda pos: floor(pos * 100)
coarse_velocity = lambda vel: floor(pos * 10)

coarse_fns = [coarse_position, coarse_velocity, coarse_position, coarse_velocity]

offset = [1/200, 1/20, 1/200, 1/20]
bias = [1/100, 1/10, 1/100, 1/10]
bias1 = bias + offset
bias2 = bias1 + offset
bias3 = bias2 + offset 

g1 = CoarseGrid

SyntaxError: ignored